In [3]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [6]:
import os, sys
#import preprocess
from keras.optimizers import *
from keras.callbacks import *

Using TensorFlow backend.


In [10]:
! ls


datalab  drive


In [11]:
! pwd

/content


In [0]:
os.chdir("drive/Colab Notebooks")

In [0]:
import preprocess

In [22]:
itokens, otokens = preprocess.MakeS2SDict('train.txt', dict_file='train_word.txt')
Xtrain, Ytrain = preprocess.MakeS2SData('train.txt', itokens, otokens, h5_file='train.h5')
Xtest, Ytest = preprocess.MakeS2SData('test.txt', itokens, otokens, h5_file='test.h5')

loading train_word.txt
loading train.h5
loading test.h5


In [23]:
print('seq 1 words:', itokens.num())
print('seq 2 words:', otokens.num())
print('train shapes:', Xtrain.shape, Ytrain.shape)
print('test shapes:', Xtest.shape, Ytest.shape)

seq 1 words: 3369
seq 2 words: 3665
train shapes: (29000, 43) (29000, 47)
test shapes: (1014, 34) (1014, 39)


In [0]:
from transformer import Transformer, LRSchedulerPerStep, LRSchedulerPerEpoch

In [0]:
d_model = 256
s2s = Transformer(itokens, otokens, len_limit=70, d_model=d_model, d_inner_hid=512, \
				   n_head=4, d_k=64, d_v=64, layers=2, dropout=0.1)

In [0]:
lr_scheduler = LRSchedulerPerStep(d_model, 4000)   # there is a warning that it is slow, however, it's ok.
#lr_scheduler = LRSchedulerPerEpoch(d_model, 4000, Xtrain.shape[0]/64)  # this scheduler only update lr per epoch
model_saver = ModelCheckpoint('en2de.model.h5', save_best_only=True, save_weights_only=True)

In [28]:
s2s.compile(Adam(0.001, 0.9, 0.98, epsilon=1e-9))
s2s.model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, None)         0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, None)         0           lambda_1[0][0]                   
__________________________________________________________________________________________________
lambda_3 (

/content/drive/Colab Notebooks/transformer.py:281: UserWarning: Output "lambda_58" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "lambda_58" during training.
  self.model.compile(optimizer, None)


In [29]:
try: s2s.model.load_weights('en2de.model.h5')
except: print('\n\nnew model')
s2s.model.fit([Xtrain, Ytrain], None, batch_size=64, epochs=30, \
				validation_data=([Xtest, Ytest], None), \
				callbacks=[lr_scheduler, model_saver])



new model
Train on 29000 samples, validate on 1014 samples
Epoch 1/30
29000/29000 [==============================] - 67s 2ms/step - loss: 6.1502 - ppl: 890.2581 - accu: 0.1265 - val_loss: 4.7976 - val_ppl: 121.7710 - val_accu: 0.1811
Epoch 2/30
 3776/29000 [==>...........................] - ETA: 55s - loss: 4.7022 - ppl: 110.6843 - accu: 0.2232

29000/29000 [==============================] - 63s 2ms/step - loss: 4.3496 - ppl: 79.7786 - accu: 0.2759 - val_loss: 3.9087 - val_ppl: 50.0955 - val_accu: 0.3166
Epoch 3/30
14080/29000 [=============>................] - ETA: 32s - loss: 3.7887 - ppl: 44.6181 - accu: 0.3325

29000/29000 [==============================] - 63s 2ms/step - loss: 3.6167 - ppl: 38.0882 - accu: 0.3581 - val_loss: 3.2698 - val_ppl: 26.5147 - val_accu: 0.4079
Epoch 4/30
17920/29000 [=================>............] - ETA: 23s - loss: 3.0844 - ppl: 22.0820 - accu: 0.4354

29000/29000 [==============================] - 63s 2ms/step - loss: 2.9985 - ppl: 20.3628 - accu: 0.4492 - val_loss: 2.6725 - val_ppl: 14.6118 - val_accu: 0.4981
Epoch 5/30
19328/29000 [==================>...........] - ETA: 20s - loss: 2.6006 - ppl: 13.5748 - accu: 0.5038

29000/29000 [==============================] - 63s 2ms/step - loss: 2.5476 - ppl: 12.9076 - accu: 0.5116 - val_loss: 2.3214 - val_ppl: 10.3186 - val_accu: 0.5457
Epoch 6/30
19904/29000 [===================>..........] - ETA: 19s - loss: 2.2403 - ppl: 9.4648 - accu: 0.5532

...

29000/29000 [==============================] - 62s 2ms/step - loss: 0.7590 - ppl: 2.1415 - accu: 0.7908 - val_loss: 1.3857 - val_ppl: 4.0282 - val_accu: 0.7100
Epoch 27/30
20416/29000 [====================>.........] - ETA: 18s - loss: 0.7263 - ppl: 2.0721 - accu: 0.7981

29000/29000 [==============================] - 63s 2ms/step - loss: 0.7381 - ppl: 2.0969 - accu: 0.7955 - val_loss: 1.3996 - val_ppl: 4.0853 - val_accu: 0.7035
Epoch 28/30
20416/29000 [====================>.........] - ETA: 18s - loss: 0.7030 - ppl: 2.0240 - accu: 0.8034

29000/29000 [==============================] - 63s 2ms/step - loss: 0.7171 - ppl: 2.0533 - accu: 0.8000 - val_loss: 1.4099 - val_ppl: 4.1240 - val_accu: 0.7068
Epoch 29/30
20416/29000 [====================>.........] - ETA: 18s - loss: 0.6856 - ppl: 1.9887 - accu: 0.8066

29000/29000 [==============================] - 62s 2ms/step - loss: 0.6956 - ppl: 2.0090 - accu: 0.8042 - val_loss: 1.4008 - val_ppl: 4.0858 - val_accu: 0.7057
Epoch 30/30
20416/29000 [====================>.........] - ETA: 18s - loss: 0.6686 - ppl: 1.9550 - accu: 0.8102

29000/29000 [==============================] - 63s 2ms/step - loss: 0.6797 - ppl: 1.9773 - accu: 0.8074 - val_loss: 1.4323 - val_ppl: 4.2264 - val_accu: 0.7072
